In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [42]:
cols = ['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription', 'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount',
           'EquipmentID', 'Latitude', 'Longitude']

faults = pd.read_csv("../data/J1939Faults.csv", usecols = cols, dtype = str)
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,0,111,17,True,2,1439,38.857638,-84.626851
1,2,990360,2015-02-21 11:34:34.000,NaN,11,629,12,True,127,1439,38.857638,-84.626851
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,11,1807,2,False,127,1369,41.42125,-87.767361
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,11,1807,2,True,127,1369,41.421018,-87.767361
4,5,990416,2015-02-21 11:39:41.000,NaN,0,4364,17,False,2,1674,38.416481,-89.442638


In [43]:
# convert dtypes as needed
# some columns have numeric-appearing values but they're not continuous variables, leaving them as strings
faults['EventTimeStamp'] = pd.to_datetime(faults['EventTimeStamp'])
faults['activeTransitionCount'] = faults['activeTransitionCount'].astype(int)

In [44]:
# time of day for faults could be interesting, split event column to date and timestamp columns
faults['event_date'] = faults['EventTimeStamp'].dt.date
faults['event_time'] = faults['EventTimeStamp'].dt.time

In [45]:
# sort df by equipment, spn, timestamp
faults = faults.copy()
faults = faults.sort_values(by = ['EquipmentID', 'spn', 'EventTimeStamp'])

# get index for first row in each group, yields list of integers
first_index = faults.groupby(['EquipmentID', 'spn']).head(1).index

# check first_index for rows where active == False
drop_index = first_index[faults.loc[first_index, 'active'] == 'False']

# drop rows where first index is an active False row
faults = faults.drop(drop_index)

# now this should work as expected
faults['false_eventTimeStamp'] = faults.sort_values(by = ['EventTimeStamp']).groupby(by = ['EquipmentID', 'spn'])['EventTimeStamp'].shift(-1)

In [46]:
# test to reveal if the process worked (this group previously had a False as the first timestamp)
faults.loc[(faults['EquipmentID'] == '1369') & (faults['spn'] == '1807')].sort_values(by = ['EventTimeStamp'])

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,event_date,event_time,false_eventTimeStamp
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,11,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21,11:35:33,2015-02-21 11:57:37
31,32,990702,2015-02-21 11:57:37,Incorrect Data Steering Wheel Angle,11,1807,2,False,127,1369,41.42787,-87.756759,2015-02-21,11:57:37,2015-02-21 12:13:47
49,50,990999,2015-02-21 12:13:47,Incorrect Data Steering Wheel Angle,11,1807,2,True,127,1369,41.431574,-87.758981,2015-02-21,12:13:47,2015-02-21 18:26:34
421,422,995975,2015-02-21 18:26:34,Incorrect Data Steering Wheel Angle,11,1807,2,False,127,1369,38.330833,-85.757037,2015-02-21,18:26:34,2015-02-21 18:26:37
422,423,995979,2015-02-21 18:26:37,Incorrect Data Steering Wheel Angle,11,1807,2,True,127,1369,38.330833,-85.757083,2015-02-21,18:26:37,2015-02-21 18:32:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6426,6427,1088496,2015-02-26 12:14:25,Incorrect Data Steering Wheel Angle,11,1807,2,True,127,1369,36.936018,-86.50726800000001,2015-02-26,12:14:25,2015-02-26 13:00:12
6434,6435,1089226,2015-02-26 13:00:12,Incorrect Data Steering Wheel Angle,11,1807,2,False,127,1369,36.717453000000006,-86.525,2015-02-26,13:00:12,2015-02-26 13:11:23
6437,6438,1089547,2015-02-26 13:11:23,Incorrect Data Steering Wheel Angle,11,1807,2,True,127,1369,36.718148,-86.525324,2015-02-26,13:11:23,2015-02-26 16:05:38
6492,6493,1094483,2015-02-26 16:05:38,Incorrect Data Steering Wheel Angle,11,1807,2,False,127,1369,35.679212,-88.745046,2015-02-26,16:05:38,2015-02-26 16:05:41


In [47]:
# mask for each service station, lat and long to 2 decimal places
station_1 = (faults['Latitude'].str.contains('36.06')) & (faults['Longitude'].str.contains('86.43'))
station_2 = (faults['Latitude'].str.contains('35.58')) & (faults['Longitude'].str.contains('86.44'))
station_3 = (faults['Latitude'].str.contains('36.19')) & (faults['Longitude'].str.contains('83.17'))

# dataframe without faults associated with service locations
faults_nonservice = faults[~(station_1 | station_2 | station_3)]

### next steps
- add date and time breakout columns (year, month, date, weekday, hour, day segment (divide 24 hours into quadrants))
- merge diagnostic data with filtered faults data (inner join)
- figure out imputing methods
- for models, imput after train/test split
- do some eda on the data

Instead of dropping all service rows, keep the first one per vehicle on a particular day. New column to mark if the row represents service or fault (boolean). Can give time etc since last service.

In [48]:
faults_nonservice = faults_nonservice.copy()

faults_nonservice['event_year'] = faults_nonservice['EventTimeStamp'].dt.year
faults_nonservice['event_month'] = faults_nonservice['EventTimeStamp'].dt.month
faults_nonservice['event_day'] = faults_nonservice['EventTimeStamp'].dt.day
faults_nonservice['event_dayofweek'] = faults_nonservice['EventTimeStamp'].dt.dayofweek
faults_nonservice['event_dayname'] = faults_nonservice['EventTimeStamp'].dt.day_name()
faults_nonservice['event_hour'] = faults_nonservice['EventTimeStamp'].dt.hour
faults_nonservice['event_time_quadrant'] = faults_nonservice['EventTimeStamp'].dt.hour // 6
faults_nonservice['event_time_half'] = faults_nonservice['EventTimeStamp'].dt.hour // 12

In [49]:
faults_nonservice['event_time_quadrant'].unique()

array([1, 3, 2, 0])

In [50]:
# drop active == False (Kagon is keeping these rows so it will give us a good way to compare performance)
faults_active = faults_nonservice.loc[faults_nonservice['active'] == 'True']

In [51]:
# for each equipment id that experienced a derate, durration of time before derate for each fault
# complicated by equipment that has more than one derate event
# derates can occur at any time of day (tend to be clustered during working hours bet that's probably because there are more trucks active) so don't want to just group by day
# just to get something working, go on date for now
faults_active = faults_active.copy()
faults_active['derate_full'] = np.where(faults_active.groupby(['EquipmentID', 'event_date'])['spn'].transform(lambda x: (x == '5246').any()), 'True', 'False')
faults_active['derate_partial'] = np.where(faults_active.groupby(['EquipmentID', 'event_date'])['spn'].transform(lambda x: (x == '1569').any()), 'True', 'False')

# come back to this later when my brain works
# faults_active['derates_all'] = np.where(faults_active.groupby(['EquipmentID', 'event_date'])['spn'].transform(lambda x: (x == '5246').any()), 'True', 'False')

In [52]:
# additional/alternative target columns to narrow focus to quadrant or half day - event_time_quadrant and event_time_half
faults_active['derate_full_quadrant'] = np.where(faults_active.groupby(['EquipmentID', 'event_date', 'event_time_quadrant'])['spn'].transform(lambda x: (x == '5246').any()), 'True', 'False')
faults_active['derate_full_half'] = np.where(faults_active.groupby(['EquipmentID', 'event_date', 'event_time_half'])['spn'].transform(lambda x: (x == '5246').any()), 'True', 'False')

In [53]:
# partial derate occurring on the same day as a full derate - incorporate this as a signal boost?
# come back to this after initial model
faults_active.loc[(faults_active['spn'] == '1569') & (faults_active['derate_full'] == 'True')]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant,event_time_half,derate_full,derate_partial,derate_full_quadrant,derate_full_half
996749,1032907,54267109,2018-07-06 04:53:05,Condition Exists Engine Protection Torque Derate,0,1569,31,True,2,105349576,...,6,4,Friday,4,0,0,True,True,False,True
5714,5715,1070647,2015-02-25 13:53:08,Condition Exists Engine Protection Torque Derate,0,1569,31,True,1,1329,...,25,2,Wednesday,13,2,1,True,True,True,True
82873,85259,2919536,2015-06-12 08:24:15,Condition Exists Engine Protection Torque Derate,0,1569,31,True,1,1339,...,12,4,Friday,8,1,0,True,True,False,False
82607,84993,2915364,2015-06-12 03:57:49,Condition Exists Engine Protection Torque Derate,0,1569,31,True,1,1366,...,12,4,Friday,3,0,0,True,True,False,True
83619,86005,2932223,2015-06-12 19:26:10,Condition Exists Engine Protection Torque Derate,0,1569,31,True,1,1366,...,12,4,Friday,19,3,1,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125718,1177138,93351982,2019-07-09 00:11:00,Condition Exists Engine Protection Torque Derate,0,1569,31,True,1,1991,...,9,1,Tuesday,0,0,0,True,True,True,True
964660,995168,45003043,2018-04-07 02:07:44,Condition Exists Engine Protection Torque Derate,0,1569,31,True,2,2007,...,7,5,Saturday,2,0,0,True,True,True,True
794693,815232,18884125,2017-06-24 10:42:36,Condition Exists Engine Protection Torque Derate,0,1569,31,True,1,2009,...,24,5,Saturday,10,1,0,True,True,False,False
1139261,1192545,100080700,2019-08-30 18:05:29,Condition Exists Engine Protection Torque Derate,0,1569,31,True,1,2021,...,30,4,Friday,18,3,1,True,True,True,True


In [54]:
diagnostics = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

In [55]:
# pivot on distinct values in 'Name'
diag_pivot = diagnostics.pivot(index = ['FaultId'], columns = ['Name'], values = ['Value'])
diag_pivot.columns = diag_pivot.columns.droplevel()

In [56]:
diag_pivot = diag_pivot.reset_index().rename_axis(None, axis = 1)


In [57]:
diag_pivot['FaultId'] = diag_pivot['FaultId'].astype(str)

In [58]:
# replace comma with period in all columns
diag_pivot = diag_pivot.replace(',', '.', regex = True)

In [59]:
# merge faults_active to diag_pivot on RecordID = FaultId
# how = left (keeps active True from faults and drops additional rows from diagnostics)
fault_diag = pd.merge(faults_active, diag_pivot, how = 'left', left_on = 'RecordID', right_on = 'FaultId')

In [60]:
fault_diag.isnull().sum()
# total rows ~550K
# rows that definitely don't have enough values to be useful: ServiceDistance, SwitchedBatteryVoltage
# extremely unlikely to be useful (1/3 to over half missing): FuelTemperature, ParkingBrake, Throttle
# iffy but possible to imput with some degree of meaning: FuelLevel, AcceleratorPedal
# quite a few are around 20-30K missing values

RecordID                          0
ESS_Id                            0
EventTimeStamp                    0
eventDescription              29996
ecuSource                         0
spn                               0
fmi                               0
active                            0
activeTransitionCount             0
EquipmentID                       0
Latitude                          0
Longitude                         0
event_date                        0
event_time                        0
false_eventTimeStamp            934
event_year                        0
event_month                       0
event_day                         0
event_dayofweek                   0
event_dayname                     0
event_hour                        0
event_time_quadrant               0
event_time_half                   0
derate_full                       0
derate_partial                    0
derate_full_quadrant              0
derate_full_half                  0
FaultId                     

In [61]:
# drop columns that are unlikely to be meaningful
fault_diag = fault_diag.drop(['ServiceDistance', 'SwitchedBatteryVoltage', 'ParkingBrake', 'FaultId'], axis = 1)

In [62]:
# convert data types
# add more as the need arises
convert_dict = {'AcceleratorPedal' : float,
                'BarometricPressure' : float,
                'CruiseControlSetSpeed' : float,
                'DistanceLtd' : float,
                'EngineCoolantTemperature': float,
                'EngineLoad' : float,
                'EngineOilPressure' : float,
                'EngineOilTemperature': float,
                'EngineRpm': float,
                'EngineTimeLtd': float,
                'FuelLevel' : float,
                'FuelLtd' : float,
                'FuelRate' : float,
                'FuelTemperature' : float,
                'IntakeManifoldTemperature' : float,
                'LampStatus' : float,
                'Speed' : float,
                'Throttle' : float,
                'TurboBoostPressure' : float}

fault_diag = fault_diag.astype(convert_dict)

In [63]:
fault_diag.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure
0,1038243,55748536,2018-07-20 09:31:33,High (Severity Medium) J1939 Network #2,49,1231,16,True,2,0105406655,...,NaN,12311.606397,1.651080,NaN,True,138.2,255.0,5.203984,100.0,5.80
1,366301,7171498,2016-01-31 07:12:25,NaN,11,629,12,True,127,105301976,...,NaN,NaN,NaN,NaN,True,NaN,1279.0,NaN,NaN,NaN
2,1141606,81618595,2019-03-27 08:10:52,Low (Severity Medium) Engine Coolant Level,0,111,18,True,1,105338729,...,44.0,14987.537198,1.083109,NaN,True,59.0,2047.0,0.000000,100.0,0.29
3,1142275,81843663,2019-03-29 06:08:56,Low (Severity Medium) Engine Coolant Level,0,111,18,True,3,105338729,...,52.8,15098.753632,0.990648,NaN,True,73.4,2047.0,0.000000,100.0,0.00
4,1149647,83749925,2019-04-16 12:27:33,Low (Severity Medium) Engine Coolant Level,0,111,18,True,4,105338729,...,62.0,15782.959247,0.039626,NaN,True,95.0,2047.0,67.525570,100.0,3.48


In [64]:
# adding a concat column for spn and fmi
fault_diag['spn_fmi'] = fault_diag['spn'] + '_' + fault_diag['fmi']

Rob's idea to categorize diagnostics based on normal operating parameters, so categorical column with low, normal, high.
To implement: dictionary with diagnostic name as key and tuple or list of lower and upper limits of normal range
Loop through dictionary, create new column name based on diagnostic and assign low if below lower limit, normal if below upper limit, else high.

In [65]:
fault_diag.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi
0,1038243,55748536,2018-07-20 09:31:33,High (Severity Medium) J1939 Network #2,49,1231,16,True,2,0105406655,...,12311.606397,1.651080,NaN,True,138.2,255.0,5.203984,100.0,5.80,1231_16
1,366301,7171498,2016-01-31 07:12:25,NaN,11,629,12,True,127,105301976,...,NaN,NaN,NaN,True,NaN,1279.0,NaN,NaN,NaN,629_12
2,1141606,81618595,2019-03-27 08:10:52,Low (Severity Medium) Engine Coolant Level,0,111,18,True,1,105338729,...,14987.537198,1.083109,NaN,True,59.0,2047.0,0.000000,100.0,0.29,111_18
3,1142275,81843663,2019-03-29 06:08:56,Low (Severity Medium) Engine Coolant Level,0,111,18,True,3,105338729,...,15098.753632,0.990648,NaN,True,73.4,2047.0,0.000000,100.0,0.00,111_18
4,1149647,83749925,2019-04-16 12:27:33,Low (Severity Medium) Engine Coolant Level,0,111,18,True,4,105338729,...,15782.959247,0.039626,NaN,True,95.0,2047.0,67.525570,100.0,3.48,111_18


In [66]:
fault_diag.loc[(fault_diag['spn'] == '5246')]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi
481,1032993,54295225,2018-07-06 09:42:48,NaN,0,5246,0,True,1,105349576,...,36865.606115,0.752893,NaN,True,145.4,22527.0,0.0,100.0,0.29,5246_0
653,1004301,47074665,2018-04-27 06:07:55,NaN,0,5246,0,True,1,105427203,...,39869.506518,1.030274,32.0,True,69.8,22527.0,0.0,100.0,0.00,5246_0
1001,5714,1070646,2015-02-25 13:53:08,NaN,0,5246,0,True,1,1329,...,NaN,NaN,NaN,True,NaN,22527.0,NaN,NaN,NaN,5246_0
2224,85811,2928718,2015-06-12 15:35:22,NaN,0,5246,0,True,1,1339,...,NaN,NaN,NaN,True,NaN,22527.0,NaN,NaN,NaN,5246_0
10200,85158,2918010,2015-06-12 06:13:27,NaN,0,5246,0,True,1,1366,...,77645.053266,0.792519,NaN,True,134.6,22527.0,0.0,NaN,0.58,5246_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543946,1078306,68092283,2018-11-12 10:43:03,NaN,49,5246,19,True,37,305,...,44664.229262,1.439742,91.4,True,95.0,255.0,0.0,0.0,0.58,5246_19
543947,1080013,68460210,2018-11-15 13:00:44,NaN,49,5246,19,True,38,305,...,44666.210552,0.488720,84.2,True,150.8,255.0,0.0,0.0,0.00,5246_19
543948,1080396,68549599,2018-11-16 10:25:51,NaN,49,5246,19,True,40,305,...,44671.361907,0.528346,64.4,True,123.8,255.0,0.0,0.0,0.29,5246_19
543949,1082051,68976239,2018-11-20 14:37:43,NaN,49,5246,19,True,42,305,...,44678.890811,1.426534,69.8,True,102.2,255.0,0.0,0.0,0.87,5246_19


In [67]:
# fuel to gas ratio
# (turboboost + barometric)  / fuel rate
fault_diag['fuel_gas_ratio'] = fault_diag['FuelRate'] / (fault_diag['TurboBoostPressure'] + fault_diag['BarometricPressure'])

In [68]:
# engine oil temp - coolant temp (if diff greater than 25 could be a problem)
fault_diag['oil_cool_temp_diff'] = fault_diag['EngineOilTemperature'] - fault_diag['EngineCoolantTemperature']

In [69]:
# # replace with real operating parameters
diag_cols = {
    # 'AcceleratorPedal' : [5, 10],
    # 'BarometricPressure' : [5, 10],
    # 'CruiseControlSetSpeed' : [5, 10],
    # 'DistanceLtd' : [5, 10],
    # 'EngineCoolantTemperature': [5, 10],
    # 'EngineLoad' : [5, 10],
    # 'EngineOilPressure' : [5, 10],
    # 'EngineOilTemperature': [5, 10],
    # 'EngineRpm': [5, 10],
    # 'EngineTimeLtd': [5, 10],
    # 'FuelLevel' : [5, 10],
    # 'FuelLtd' : [5, 10],
    # 'FuelRate' : [5, 10],
    # 'FuelTemperature' : [5, 10],
    # 'IntakeManifoldTemperature' : [5, 10],
    # 'LampStatus' : [5, 10],
    # 'Speed' : [5, 10],
    # 'Throttle' : [5, 10],
    # 'TurboBoostPressure' : [5, 10],
    'oil_cool_temp_diff' : [0, 25]
    }

In [70]:
for col, bounds in diag_cols.items():
    low_bound, high_bound = bounds
    new_col = f'{col}_cat'
    fault_diag[new_col] = pd.cut(fault_diag[col], bins = [-float('inf'), low_bound, high_bound, float('inf')], labels = ['low', 'normal', 'high'])

In [71]:
# ID column to group rows by date and equipment id so they can easily be evauated as a unit
fault_diag['equip_date_id'] = fault_diag['EquipmentID'].astype(str) + '_' + fault_diag['event_date'].astype(str)

In [72]:
fault_diag.to_csv('../data/fault_diag.csv', index = False)

### target:
- group by equipment ID and sort by event datetime
- if equipment had derate (fault id) boolean column true for records within a certain timeframe prior
- looking for a signal with enough time to get the truck to a service location

### predictors:
- fault codes, diagnostics (downside is that a lot of values are missing, but they seem to be more present with active true)
- might not be value in looking at active false (Tomo's group only looked at active true)

### do this as part of data cleaning:
- pull timestamp from active false to get duration of active true (there should be paired rows based on fault code, different lights for different faults)
- do this before dropping service locations (don't want to miss active false which happened as a result of service)

### other thoughts:
- Code P0606 tends to be set when a PCM/ECM has failed. Depending on component condition and the make and model of a vehicle, it may be possible to resolve an internal integrity fault in the PCM/ECM by upgrading or re-flashing the control module.
- since it's a fault with the monitoring device itself, might not have any predictive power
- how many are there?
- if we get rid of this (and other faults not actually related to the engine) might reduce noise in the dataset?
- since there are active true with no diagnostics, could the rows with null diagnostics be faults like this one that don't have anything to do with the engine?